# 🧬 Geometry-Complete Equivariant Diffusion Model
## De Novo Drug Design Training Notebook

**Requirements:** GPU Runtime (T4), ~55GB disk space

## Cell 1: Check GPU & Disk Space

In [ ]:
import torch
import shutil

# GPU check
print(f"PyTorch: {torch.__version__}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("❌ No GPU! Runtime > Change runtime type > GPU")

# Disk check
total, used, free = shutil.disk_usage('/')
print(f"\n💾 Disk: {free / 1e9:.1f} GB free of {total / 1e9:.1f} GB")
if free / 1e9 < 55:
    print("⚠️ Less than 55GB free - may not fit full dataset")

## Cell 2: Install Dependencies

In [ ]:
!pip install -q torch-geometric rdkit scipy numpy pyyaml tqdm wandb
from rdkit import Chem
print("✅ All dependencies installed")

## Cell 3: Clone Repository

In [ ]:
!git clone https://github.com/Nethrananda21/geom_diffusion.git
%cd geom_diffusion
!git pull origin master

## Cell 4: Download & Extract Dataset (Stream Mode - Saves 50GB!)

⚠️ This downloads and extracts simultaneously, never storing the full 50GB archive.

**Skip this cell to use synthetic data for quick testing.**

In [ ]:
import os

# Create data directory
os.makedirs('data', exist_ok=True)

print("📥 Downloading and extracting CrossDocked2020 (streaming mode)...")
print("   This takes 30-60 minutes. Do NOT close the browser.")

# Stream download + extract (no intermediate .tgz file saved)
!wget -O - --no-check-certificate http://bits.csb.pitt.edu/files/crossdock2020/CrossDocked2020_v1.3.tgz 2>/dev/null | tar -xzf - -C ./data/

# Verify extraction
if os.path.exists('data/CrossDocked2020'):
    print("\n✅ Dataset extracted successfully to data/CrossDocked2020/")
    !du -sh data/CrossDocked2020
else:
    # Check if extracted with different folder name
    !ls -la data/
    print("\n⚠️ Check folder name above - may need to rename")

## Cell 5: Preprocess Dataset (Creates .pkl files)

In [ ]:
# Find the extracted folder (might have different name)
import os
from pathlib import Path

data_dir = None
for folder in ['CrossDocked2020', 'crossdocked2020', 'CrossDocked2020_v1.3']:
    if Path(f'data/{folder}').exists():
        data_dir = f'data/{folder}'
        break

if data_dir:
    print(f"📂 Found dataset at: {data_dir}")
    print("⏳ Preprocessing (this takes 10-20 minutes)...")
    !python preprocess_crossdocked.py \
        --data_dir {data_dir} \
        --output_dir ./data/crossdocked \
        --config configs/debug_t4.yaml
    print("\n✅ Preprocessing complete")
    !ls -la data/crossdocked/
else:
    print("❌ Dataset folder not found. Check Cell 4 output.")
    !ls -la data/

## Cell 6: Delete Old Cache (CRITICAL!)

In [ ]:
import shutil
from pathlib import Path

# Delete synthetic cache
cache_dir = Path('data/cache')
if cache_dir.exists():
    shutil.rmtree(cache_dir)
    print("🗑️ Deleted old cache")
else:
    print("ℹ️ No cache to delete")

# Verify real data
train_pkl = Path('data/crossdocked/train_data.pkl')
val_pkl = Path('data/crossdocked/val_data.pkl')

if train_pkl.exists() and val_pkl.exists():
    print(f"\n✅ Ready to train on REAL data:")
    print(f"   - {train_pkl} ({train_pkl.stat().st_size / 1e6:.1f} MB)")
    print(f"   - {val_pkl} ({val_pkl.stat().st_size / 1e6:.1f} MB)")
else:
    print("\n⚠️ Real data NOT found - will use SYNTHETIC")

## Cell 7: Configure Training

In [ ]:
import yaml

with open('configs/debug_t4.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Colab optimizations
config['training']['max_epochs'] = 10  # Quick test
config['hardware']['num_workers'] = 2  # Colab CPU limit

with open('configs/debug_t4.yaml', 'w') as f:
    yaml.dump(config, f)

print(f"✅ Config: batch={config['training']['batch_size']}, epochs={config['training']['max_epochs']}")

## Cell 8: Start Training 🚀

In [ ]:
# Watch logs - should say "Loading preprocessed data" NOT "SYNTHETIC"
!python train.py --config configs/debug_t4.yaml --checkpoint_dir ./checkpoints

## Cell 9: Resume Training (If Interrupted)

In [ ]:
# Uncomment to resume
# !python train.py --config configs/debug_t4.yaml --resume ./checkpoints/best_model.pt

## Cell 10: Download Trained Model

In [ ]:
from google.colab import files
from pathlib import Path

ckpt = Path('checkpoints/best_model.pt')
if ckpt.exists():
    files.download(str(ckpt))
    print("✅ Model downloaded")
else:
    print("❌ No checkpoint yet")